TODO
* Merge Tables 2,4,10 to work together
* Create loop for full season
* Add season and episode to front of data frame

Someday:
* Fix Italics:
    * Work: 
        * italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
    * Not work: 
        * italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
        
Learning: 
* Look for specific class types in BeautifulSoup. The fact that the transcript was formated as a *wikitable* made life much easier

In [1]:
# imports

#standard
import pandas as pd
#import random
from datetime import datetime
from pandasql import sqldf
pysqldf = lambda q: sqldf(q,globals())

#Viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Data Pulls
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
#Get Episode titles for Season 1

In [2]:
url_base = 'https://avatar.fandom.com/wiki/Avatar_Wiki:Transcripts'
response = requests.get(url_base)

In [3]:
soup = BeautifulSoup(response.content, "html.parser")

tables = soup.find_all("table")

print(f"Found {len(tables)} tables on the page.")

Found 90 tables on the page.


In [4]:
# Table 2: Book 1 Water
# Table 4: Book 2 Earth
# Table 10: Book 3 Fire

# if len(tables) > 0:
#     print("First table:")
#     print(tables[10].prettify())

In [5]:
book_list = [2,5,10]

In [6]:
episode_list = []

for book in book_list:
    specific_table = soup.find_all("table")[book]  # Replace 0 with the index of the table you want
    links = specific_table.find_all("a")
    for link in links:
        episode_list.append(link.get("href"))

In [7]:
episode_list_filtered = [episode for episode in episode_list if "commentary" not in episode]
#episode_list_filtered

In [8]:
#Creates Episodes URLs
base = 'https://avatar.fandom.com'
url_episode_list = []
for episode in episode_list_filtered:
    print(base + episode)
    url_episode_list.append(base + episode)

https://avatar.fandom.com/wiki/Transcript:The_Boy_in_the_Iceberg
https://avatar.fandom.com/wiki/Transcript:The_Avatar_Returns
https://avatar.fandom.com/wiki/Transcript:The_Southern_Air_Temple
https://avatar.fandom.com/wiki/Transcript:The_Warriors_of_Kyoshi
https://avatar.fandom.com/wiki/Transcript:The_King_of_Omashu
https://avatar.fandom.com/wiki/Transcript:Imprisoned
https://avatar.fandom.com/wiki/Transcript:Winter_Solstice,_Part_1:_The_Spirit_World
https://avatar.fandom.com/wiki/Transcript:Winter_Solstice,_Part_2:_Avatar_Roku
https://avatar.fandom.com/wiki/Transcript:The_Waterbending_Scroll
https://avatar.fandom.com/wiki/Transcript:Jet_(episode)
https://avatar.fandom.com/wiki/Transcript:The_Great_Divide
https://avatar.fandom.com/wiki/Transcript:The_Storm
https://avatar.fandom.com/wiki/Transcript:The_Blue_Spirit
https://avatar.fandom.com/wiki/Transcript:The_Fortuneteller
https://avatar.fandom.com/wiki/Transcript:Bato_of_the_Water_Tribe
https://avatar.fandom.com/wiki/Transcript:The_Des

### Edge Case Identifier

In [9]:
#Printe Edge Case Episodes
for episode_number, url in enumerate(url_episode_list):
    
    split_text = url.split(':', 2)
    episode_title = split_text[2]

    episode_number += 1
    # print(episode_number)
    # print(episode_title)
    # print(url)
    
    response = requests.get(url)
    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the tables with the class "wikitable"
    wikitable_tables = soup.find_all("table", class_="wikitable")
    num_tables = len(wikitable_tables)
    
    if num_tables >= 2:
            print(episode_number)
            print(episode_title)
            print(url)
            print(f"Number of Tables: {len(wikitable_tables)}")

1
The_Boy_in_the_Iceberg
https://avatar.fandom.com/wiki/Transcript:The_Boy_in_the_Iceberg
Number of Tables: 2
2
The_Avatar_Returns
https://avatar.fandom.com/wiki/Transcript:The_Avatar_Returns
Number of Tables: 2
15
Bato_of_the_Water_Tribe
https://avatar.fandom.com/wiki/Transcript:Bato_of_the_Water_Tribe
Number of Tables: 2
35
The_Tales_of_Ba_Sing_Se
https://avatar.fandom.com/wiki/Transcript:The_Tales_of_Ba_Sing_Se
Number of Tables: 6
48
The_Puppetmaster
https://avatar.fandom.com/wiki/Transcript:The_Puppetmaster
Number of Tables: 2


# Edge Case Exploration

#url = 'https://avatar.fandom.com/wiki/Transcript:The_Boy_in_the_Iceberg'
# Index 1 | 2nd Table

#url = 'https://avatar.fandom.com/wiki/Transcript:The_Avatar_Returns'
# Index 1 2nd Table 

# url = 'https://avatar.fandom.com/wiki/Transcript:Bato_of_the_Water_Tribe'
# Index 0 1st Table 
# 2nd Table contains a deleted scene

#url = 'https://avatar.fandom.com/wiki/Transcript:The_Tales_of_Ba_Sing_Se'
# Index 0 The Tale of Toph and Katara
# Index 1 The Tale of Iroh
# Index 2 The Tale of Aang
# Index 3 The Tale of Sokka 
# Index 4 THe Tale of Zuko

#url = 'https://avatar.fandom.com/wiki/Transcript:The_Puppetmaster'
# Index 1 2nd Table 
# Removes the "previously on Avatar"
    
response = requests.get(url)
ep_soup = BeautifulSoup(response.content, "html.parser")
tables = ep_soup.select("table.wikitable")

#print(tables)

data = []

print(f"Number of Tables: {len(tables)}")
print()

desired_table = tables[:4]  
# Find all rows within the 2nd Table that holds the transcript
rows = desired_table.select("tbody tr")

for row in rows:
    character_cell = row.find('th')  # Find the <th> for character names
    dialogue_cell = row.find('td')    # Find the <td> for character's dialogue

    if character_cell and dialogue_cell:  # Ensure both character & dialogue are present (thus skipping pure screen directions)
        character = character_cell.get_text(strip=True)

        # Replace <i> tags with their text (ITALICS CAUSING PROBLEMS)
        for italics in dialogue_cell.find_all('i'):
            # Get the text inside <i>, append '123' and replace the <i> with this text
            italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
            italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text

        dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics

        data.append({'character': character, 'dialogue': dialogue})

        df = pd.DataFrame(data)

#Cleaning
df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()

#Word Count
df['word_count'] = df['dialogue'].str.split().apply(len)

df

In [ ]:
#Tales of Ba Sing Se - Edge Case Exploration
url = 'https://avatar.fandom.com/wiki/Transcript:The_Tales_of_Ba_Sing_Se'
# Index 0 The Tale of Toph and Katara
# Index 1 The Tale of Iroh
# Index 2 The Tale of Aang
# Index 3 The Tale of Sokka 
# Index 4 THe Tale of Zuko
    
response = requests.get(url)
ep_soup = BeautifulSoup(response.content, "html.parser")
tables = ep_soup.select("table.wikitable")

#print(tables)

data = []

# Iterate over the first 5 tables (0 to 4)
for table in tables[:5]:  
    # Find all rows within each table
    rows = table.select("tbody tr")

    for row in rows:
        character_cell = row.find('th')  # Find the <th> for character names
        dialogue_cell = row.find('td')   # Find the <td> for character's dialogue

        if character_cell and dialogue_cell:  # Ensure both character & dialogue are present
            character = character_cell.get_text(strip=True)

            # Process the <i> tags within the dialogue
            for italics in dialogue_cell.find_all('i'):
                # Prepend and append '12345' to italicized text
                italic_text_with_suffix = '12345' + italics.get_text(strip=True) + '12345'
                italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text

            # Get the full dialogue text after modifications
            dialogue = dialogue_cell.get_text(strip=True)

            # Append the data to the list
            data.append({'character': character, 'dialogue': dialogue})

# Convert the collected data into a DataFrame
df = pd.DataFrame(data)

#Cleaning
df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()

#Word Count
df['word_count'] = df['dialogue'].str.split().apply(len)

pd.set_option('display.max_rows', None)

df

In [ ]:
pd.set_option('display.max_rows', None)
#pd.reset_option('display.max_rows')
df.head(195)

In [ ]:
#Non Edge Case Loop

In [ ]:
#IDentify problem of if intro is said

### Full Season Edge Case + Non Edge Case (...except Tales of Ba Sing Se) Loop

In [11]:
#episodes, so technically need to subtract 1 from each to get the edge case
edge_case_episodes = [1, #The_Boy_in_the_Iceberg
                      2, #The_Avatar_Returns
                      #15, #Bato_of_the_Water_Tribe - Use table 1, index 0 to remove deleted scence
                      35, #The_Tales_of_Ba_Sing_Se
                      48 #The_Puppetmaster
                     ]

#Subtract 1 from each episode to convert episode_number --> index
edge_case_episodes = [episode - 1 for episode in edge_case_episodes]

In [12]:
edge_case_episodes

[0, 1, 34, 47]

In [ ]:
### WORKING

In [18]:
#Catches all edge cases excepct Tales of Ba Sing Se
# Can't get it to work.. .

transcript_df = pd.DataFrame()

# Loop through each episode
# Can expand to more episodes
for episode_number, url in enumerate(url_episode_list):
    # Splits based on Edge Cases
    
    data = []  # Initialize data list for the current episode
    
    #Use if statement to get stuff... 
    if episode_number not in edge_case_episodes:
        
        response = requests.get(url)
        ep_soup = BeautifulSoup(response.content, "html.parser")
        tables = ep_soup.select("table.wikitable")
        #print("Non Edge Case")
        
        desired_table = tables[0]
        rows = desired_table.select("tbody tr")

    if episode_number in edge_case_episodes:
            
            if episode_number != 34:
                response = requests.get(url)
                ep_soup = BeautifulSoup(response.content, "html.parser")
                tables = ep_soup.select("table.wikitable")
                
                desired_table = tables[1]
                rows = desired_table.select("tbody tr")
                
                print("Edge Case")
                print(episode_number)
                print(episode_title)
                print(url)
                print()
                
            else:
                response = requests.get(url)
                ep_soup = BeautifulSoup(response.content, "html.parser")
                tables = ep_soup.select("table.wikitable")

                print(episode_number)
                print(episode_title)
                print(url)
                print()

                print('Ba Sing Se')

                all_rows = []  # Initialize list to store all rows from all tables

                for table in tables[:5]:
                    table_rows = table.select("tbody tr")
                    all_rows.extend(table_rows)  # Append all rows to the all_rows list

                rows = all_rows  # Now, 'rows' contains all the rows from tables 0 to 4
                    
        
    split_text = url.split(':', 2)
    episode_title = split_text[2]
    episode_number += 1
    
    #print(episode_number)
    #print(episode_title)
    #print(url)
    #print()

    for row in rows:
        character_cell = row.find('th')  # Find the <th> for character names
        dialogue_cell = row.find('td')   # Find the <td> for character's dialogue

        if character_cell and dialogue_cell:  # Ensure both character & dialogue are present
            character = character_cell.get_text(strip=True)

            # Replace <i> tags with their text (handle italicized text)
            for italics in dialogue_cell.find_all('i'):
                italic_text_with_suffix = '12345' + italics.get_text(strip=True) + '12345'
                italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text

            dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics

            # Append data for this row
            data.append({'character': character, 'dialogue': dialogue})

    # Only create DataFrame if data was extracted        
    if data:
        df = pd.DataFrame(data)

        # Cleaning the dialogue column
        df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()  # Remove text in []
        df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()  # Remove text in ()
        df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()  # Replace '12345' with space

        # Add word count
        df['word_count'] = df['dialogue'].str.split().apply(len)

        # Add episode number and title
        df['episode'] = episode_number
        df['episode_title'] = episode_title  # Fix the typo

        # Concatenate with the main transcript DataFrame
        transcript_df = pd.concat([transcript_df, df], ignore_index=True)

Edge Case
0
Sozin%27s_Comet,_Part_4:_Avatar_Aang
https://avatar.fandom.com/wiki/Transcript:The_Boy_in_the_Iceberg

Edge Case
1
The_Boy_in_the_Iceberg
https://avatar.fandom.com/wiki/Transcript:The_Avatar_Returns

34
City_of_Walls_and_Secrets
https://avatar.fandom.com/wiki/Transcript:The_Tales_of_Ba_Sing_Se

Ba Sing Se
Edge Case
47
The_Runaway
https://avatar.fandom.com/wiki/Transcript:The_Puppetmaster



In [20]:
transcript_df[transcript_df['episode'] ==35].head(126)

,character,dialogue,word_count,episode,episode_title
5827,Katara,Toph! Aren't you gonna get ready for the day?,9,35,The_Tales_of_Ba_Sing_Se
5828,Toph,I'm ready.,2,35,The_Tales_of_Ba_Sing_Se
5829,Katara,You're not gonna wash up? You've got a little ...,15,35,The_Tales_of_Ba_Sing_Se
5830,Toph,"You call it dirt, I call it a healthy coating ...",12,35,The_Tales_of_Ba_Sing_Se
5831,Katara,Hmm ... You know what we need? A girl's day out!,11,35,The_Tales_of_Ba_Sing_Se
...,...,...,...,...,...
5948,Jin,Your uncle is a good teacher. I have somethin...,20,35,The_Tales_of_Ba_Sing_Se
5949,Jin,What's wrong?,2,35,The_Tales_of_Ba_Sing_Se
5950,Zuko,It's complicated. I have to go.,6,35,The_Tales_of_Ba_Sing_Se
5951,Iroh,"How was your night, Prince Zuko?",6,35,The_Tales_of_Ba_Sing_Se


In [ ]:
tran

In [ ]:
###TEST 2

In [ ]:
def process_dialogue(dialogue_cell):
    for italics in dialogue_cell.find_all('i'):
        italic_text_with_suffix = '12345' + italics.get_text(strip=True) + '12345'
        italics.replace_with(italic_text_with_suffix)
    return dialogue_cell.get_text(strip=True)

def clean_dataframe(df, episode_number, episode_title):
    df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
    df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
    df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()
    df['word_count'] = df['dialogue'].str.split().apply(len)
    df['episode'] = episode_number
    df['episode_title'] = episode_title

In [ ]:
for episode_number, url in enumerate(url_episode_list):
    data = []  # Initialize data list for the current episode

    response = requests.get(url)
    ep_soup = BeautifulSoup(response.content, "html.parser")
    tables = ep_soup.select("table.wikitable")

    # Split episode title
    split_text = url.split(':', 2)
    episode_title = split_text[2]
    episode_number += 1

    # Check if it's a non-edge case episode
    if episode_number not in edge_case_episodes:
        print("Non Edge Case")
        desired_table = tables[0]
        rows = desired_table.select("tbody tr")

    # Handle edge case episodes
    if episode_number in edge_case_episodes:
        print("Edge Case")
        
        # Special case for episode 35 (Ba Sing Se)
        if episode_number == 34:
            print('Ba Sing Se')
            for table in tables[:5]:  # Loop through tables 0 to 4
                rows = table.select("tbody tr")
                
                # Process rows from each table
                for row in rows:
                    character_cell = row.find('th')
                    dialogue_cell = row.find('td')
                    if character_cell and dialogue_cell:
                        character = character_cell.get_text(strip=True)
                        dialogue = process_dialogue(dialogue_cell)  # Extract and clean dialogue
                        data.append({'character': character, 'dialogue': dialogue})
        else:
            print(f"Processing Edge Case Episode {episode_number}")
            desired_table = tables[1]  # Only select the second table for other edge case episodes
            rows = desired_table.select("tbody tr")

    # Common row processing (both edge case and non-edge case)
    for row in rows:
        character_cell = row.find('th')
        dialogue_cell = row.find('td')

        if character_cell and dialogue_cell:
            character = character_cell.get_text(strip=True)
            dialogue = process_dialogue(dialogue_cell)
            data.append({'character': character, 'dialogue': dialogue})

    # Only create DataFrame if data was extracted
    if data:
        df = pd.DataFrame(data)
        clean_dataframe(df, episode_number, episode_title)
        transcript_df = pd.concat([transcript_df, df], ignore_index=True)

In [ ]:
transcript_df[transcript_df['episode'] ==35]

In [ ]:
transcript_df.head()

In [ ]:
transcript_df.tail()

In [ ]:
### TEST

In [ ]:
#Catches all edge cases excepct Tales of Ba Sing Se

transcript_df = pd.DataFrame()

# Loop through each episode
# Can expand to more episodes
for episode_number, url in enumerate(url_episode_list):
    # Splits based on Edge Cases
    
    data = []  # Initialize data list for the current episode
    
    #Use if statement to get stuff... 
    if episode_number not in edge_case_episodes:
        
        response = requests.get(url)
        ep_soup = BeautifulSoup(response.content, "html.parser")
        tables = ep_soup.select("table.wikitable")
        print("Non Edge Case")
        
        desired_table = tables[0]
        rows = desired_table.select("tbody tr")

    if episode_number in edge_case_episodes:
            
            if episode_number == 34:
                response = requests.get(url)
                ep_soup = BeautifulSoup(response.content, "html.parser")
                tables = ep_soup.select("table.wikitable")
                
                print('Ba Sing Se')
                for table in tables[:5]:
                    rows = table.select("tbody tr")
                    
            else:
                response = requests.get(url)
                ep_soup = BeautifulSoup(response.content, "html.parser")
                tables = ep_soup.select("table.wikitable")
                print("Edge Case")

                desired_table = tables[1]
                rows = desired_table.select("tbody tr")
        
    split_text = url.split(':', 2)
    episode_title = split_text[2]
    episode_number += 1
    print(episode_number)
    print(episode_title)
    print(url)
    print()

    #rows = desired_table.select("tbody tr")

    for row in rows:
        character_cell = row.find('th')  # Find the <th> for character names
        dialogue_cell = row.find('td')   # Find the <td> for character's dialogue

        if character_cell and dialogue_cell:  # Ensure both character & dialogue are present
            character = character_cell.get_text(strip=True)

            # Replace <i> tags with their text (handle italicized text)
            for italics in dialogue_cell.find_all('i'):
                italic_text_with_suffix = '12345' + italics.get_text(strip=True) + '12345'
                italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text

            dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics

            # Append data for this row
            data.append({'character': character, 'dialogue': dialogue})

    # Only create DataFrame if data was extracted        
    if data:
        df = pd.DataFrame(data)

        # Cleaning the dialogue column
        df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()  # Remove text in []
        df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()  # Remove text in ()
        df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()  # Replace '12345' with space

        # Add word count
        df['word_count'] = df['dialogue'].str.split().apply(len)

        # Add episode number and title
        df['episode'] = episode_number
        df['episode_title'] = episode_title  # Fix the typo

        # Concatenate with the main transcript DataFrame
        transcript_df = pd.concat([transcript_df, df], ignore_index=True)

In [ ]:
#Why not pulling in everything?
transcript_df.loc[transcript_df['episode']==35]

In [ ]:
#Catches all edge cases excepct Tales of Ba Sing Se

transcript_df = pd.DataFrame()

# Loop through each episode
# Can expand to more episodes
for episode_number, url in enumerate(url_episode_list):
    # Splits based on Edge Cases
    
    data = []  # Initialize data list for the current episode
    
    #Use if statement to get stuff... 
    if episode_number not in edge_case_episodes:
        
        response = requests.get(url)
        ep_soup = BeautifulSoup(response.content, "html.parser")
        tables = ep_soup.select("table.wikitable")
        print("Non Edge Case")
        
        desired_table = tables[0]
    
    if episode_number in edge_case_episodes:
        
        response = requests.get(url)
        ep_soup = BeautifulSoup(response.content, "html.parser")
        tables = ep_soup.select("table.wikitable")
        print("Edge Case")
        
        desired_table = tables[1]
    
    else:
        for table in tables[:5]:
            rows = table.select("tbody tr")
    
    #Ba Sing Se
    # Iterate over the first 5 tables (0 to 4)
    for table in tables[:5]:  
    # Find all rows within each table
    
        
    
    split_text = url.split(':', 2)
    episode_title = split_text[2]
    episode_number += 1
    print(episode_number)
    print(episode_title)
    print(url)
    print()

    rows = desired_table.select("tbody tr")

    for row in rows:
        character_cell = row.find('th')  # Find the <th> for character names
        dialogue_cell = row.find('td')   # Find the <td> for character's dialogue

        if character_cell and dialogue_cell:  # Ensure both character & dialogue are present
            character = character_cell.get_text(strip=True)

            # Replace <i> tags with their text (handle italicized text)
            for italics in dialogue_cell.find_all('i'):
                italic_text_with_suffix = '12345' + italics.get_text(strip=True) + '12345'
                italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text

            dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics

            # Append data for this row
            data.append({'character': character, 'dialogue': dialogue})

    # Only create DataFrame if data was extracted        
    if data:
        df = pd.DataFrame(data)

        # Cleaning the dialogue column
        df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()  # Remove text in []
        df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()  # Remove text in ()
        df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()  # Replace '12345' with space

        # Add word count
        df['word_count'] = df['dialogue'].str.split().apply(len)

        # Add episode number and title
        df['episode'] = episode_number
        df['episode_title'] = episode_title  # Fix the typo

        # Concatenate with the main transcript DataFrame
        transcript_df = pd.concat([transcript_df, df], ignore_index=True)

In [ ]:
transcript_df.head()

In [ ]:
###

In [ ]:


url = 'https://avatar.fandom.com/wiki/Transcript:The_Siege_of_the_North,_Part_2'
    
response = requests.get(url)
ep_soup = BeautifulSoup(response.content, "html.parser")
tables = ep_soup.select("table.wikitable")

print(tables)

# data = []

# if len(tables) > 1:
#     second_table = tables[1]  # Index 1 corresponds to 2nd Table which holds the transcript
# # Find all rows within the 2nd Table that holds the transcript
#     rows = second_table.select("tbody tr")

# for row in rows:
#     character_cell = row.find('th')  # Find the <th> for character names
#     dialogue_cell = row.find('td')    # Find the <td> for character's dialogue

#     if character_cell and dialogue_cell:  # Ensure both character & dialogue are present (thus skipping pure screen directions)
#         character = character_cell.get_text(strip=True)

#         # Replace <i> tags with their text (ITALICS CAUSING PROBLEMS)
#         for italics in dialogue_cell.find_all('i'):
#             # Get the text inside <i>, append '123' and replace the <i> with this text
#             italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
#             italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text

#         dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics

#         data.append({'character': character, 'dialogue': dialogue})

#         df = pd.DataFrame(data)

# #Cleaning
# df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
# df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
# df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()

# #Word Count
# df['word_count'] = df['dialogue'].str.split().apply(len)

# #Episode 
# df['episode'] = episode_number
# df['espisode_title'] = episode_title 
            
# #     print(episode_title)
    
# #     transcript_df = pd.concat([transcript_df, df], ignore_index=True)

In [ ]:
df

In [ ]:
#Try Different Edge Cases
    # Previous Problem was based on pulling the wrong table index

transcript_df = pd.DataFrame()

# Loop through each episode
# Can expand to more episodes
for episode_number, url in enumerate(url_episode_list):
    # Splits based on Edge Cases
    if episode_number not in edge_case_episodes:
    
        split_text = url.split(':', 2)
        episode_title = split_text[2]
        episode_number += 1
        #print(episode_number)
        print(episode_title)
        print(url)
        #print("Non-Edge_Case-Episode")
        print()

        response = requests.get(url)
        ep_soup = BeautifulSoup(response.content, "html.parser")
        tables = ep_soup.select("table.wikitable")

        data = []  # Initialize data list for the current episode

        #Non Edge Case Data is held in table_index 0
        desired_table = tables[0]  # Index 1 corresponds to 2nd Table which holds the transcript
        # Find all rows within the 2nd Table that holds the transcript
        rows = desired_table.select("tbody tr")

        for row in rows:
            character_cell = row.find('th')  # Find the <th> for character names
            dialogue_cell = row.find('td')   # Find the <td> for character's dialogue

            if character_cell and dialogue_cell:  # Ensure both character & dialogue are present
                character = character_cell.get_text(strip=True)

                # Replace <i> tags with their text (handle italicized text)
                for italics in dialogue_cell.find_all('i'):
                    italic_text_with_suffix = '12345' + italics.get_text(strip=True) + '12345'
                    italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text

                dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics

                # Append data for this row
                data.append({'character': character, 'dialogue': dialogue})

        # Only create DataFrame if data was extracted        
        if data:
            df = pd.DataFrame(data)

            # Cleaning the dialogue column
            df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()  # Remove text in []
            df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()  # Remove text in ()
            df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()  # Replace '12345' with space

            # Add word count
            df['word_count'] = df['dialogue'].str.split().apply(len)

            # Add episode number and title
            df['episode'] = episode_number
            df['episode_title'] = episode_title  # Fix the typo

            # Concatenate with the main transcript DataFrame
            transcript_df = pd.concat([transcript_df, df], ignore_index=True)

In [ ]:
# Full Season Loop

transcript_df = pd.DataFrame()

for episode_number, url in enumerate(url_episode_list):
    
    split_text = url.split(':', 2)
    episode_title = split_text[2]
    episode_number += 1
    print(episode_number)
    print(episode_title)
    print(url)
    print()
    
    response = requests.get(url)
    ep_soup = BeautifulSoup(response.content, "html.parser")
    tables = ep_soup.select("table.wikitable")

    data = []
    
    if len(tables) > 1:
        second_table = tables[1]  # Index 1 corresponds to 2nd Table which holds the transcript
    # Find all rows within the 2nd Table that holds the transcript
        rows = second_table.select("tbody tr")
    
    for row in rows:
        character_cell = row.find('th')  # Find the <th> for character names
        dialogue_cell = row.find('td')    # Find the <td> for character's dialogue
    
        if character_cell and dialogue_cell:  # Ensure both character & dialogue are present (thus skipping pure screen directions)
            character = character_cell.get_text(strip=True)

            # Replace <i> tags with their text (ITALICS CAUSING PROBLEMS)
            for italics in dialogue_cell.find_all('i'):
                # Get the text inside <i>, append '123' and replace the <i> with this text
                italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
                italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text
            
            dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics
            
            data.append({'character': character, 'dialogue': dialogue})

            df = pd.DataFrame(data)
            
    #Cleaning
    df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
    df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
    df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()

    #Word Count
    df['word_count'] = df['dialogue'].str.split().apply(len)

    #Episode 
    df['episode'] = episode_number
    df['espisode_title'] = episode_title 
            
#     print(episode_title)
    
#     transcript_df = pd.concat([transcript_df, df], ignore_index=True)

In [ ]:
df.head()

In [ ]:
test = transcript_df[transcript_df['episode'] == 7]
test

In [ ]:
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None) 

from tabulate import tabulate

for episode in range(1,21):
    test = transcript_df[transcript_df['episode'] == episode]
    print(test)
    #print(tabulate(test, headers='keys', tablefmt='pretty'))
    print()


In [ ]:
transcript_df.head(20)

In [ ]:
transcript_df.tail(20)

In [ ]:
#First Episode
episode_base = url_episode_list[0]
response = requests.get(episode_base)
ep_soup = BeautifulSoup(response.content, "html.parser")
#tables = ep_soup.find_all("table")

In [ ]:
tables = ep_soup.select("table.wikitable")
data = []

# Step 4: Check if there are at least 2 tables and select the second one
if len(tables) > 1:
    second_table = tables[1]  # Index 1 corresponds to the second table
    # Now, find all rows within this second table's tbody
    rows = second_table.select("tbody tr")
    
    for row in rows:
        character_cell = row.find('th')  # Find the <th> for character names
        dialogue_cell = row.find('td')    # Find the <td> for dialogue
    
        if character_cell and dialogue_cell:  # Ensure both are present
            character = character_cell.get_text(strip=True)

            # Replace <i> tags with their text (ITALICS CAUSING PROBLEMS)
            for italics in dialogue_cell.find_all('i'):
                # Get the text inside <i>, append '123' and replace the <i> with this text
                italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
                italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text
            
            dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics
            
            data.append({'character': character, 'dialogue': dialogue})

            df = pd.DataFrame(data)
            
            #Cleaning
            df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
            df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
            df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()
            
            #Word Count
            df['word_count'] = df['dialogue'].str.split().apply(len)

In [ ]:
pd.set_option('display.max_colwidth', None) 
df.head(20)

In [ ]:
pd.set_option('display.max_colwidth', None) 
df.tail(20)

In [ ]:
# Table 1 - Intro
if len(tables) > 0:
    print("Second table:")
    print()
    print(tables[1].prettify())